In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = smc.ParticleTrace(1)

In [4]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)
ssm_step.condition(trace=generative, guide=None)

In [6]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta = ssm_step((zs[:, t], mu, sigma, delta), t)

In [7]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [8]:
num_particles = 100

In [9]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [10]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [11]:
init_ssm = combinators.Model(ssm.init_ssm, phi=ssm_params, theta={})

In [12]:
inference, init_ssm_params = smc.variational_smc(num_particles, init_ssm, smc_runner, 500, 50, generative)

07/27/2018 10:01:14 Variational SMC ELBO=-1.95770723e+04 at epoch 1
07/27/2018 10:01:14 Variational SMC ELBO=-1.68765977e+04 at epoch 2
07/27/2018 10:01:15 Variational SMC ELBO=-2.20191523e+04 at epoch 3
07/27/2018 10:01:15 Variational SMC ELBO=-2.35276094e+04 at epoch 4
07/27/2018 10:01:16 Variational SMC ELBO=-2.30572070e+04 at epoch 5
07/27/2018 10:01:16 Variational SMC ELBO=-2.18067715e+04 at epoch 6
07/27/2018 10:01:17 Variational SMC ELBO=-1.73429355e+04 at epoch 7
07/27/2018 10:01:17 Variational SMC ELBO=-1.90957090e+04 at epoch 8
07/27/2018 10:01:18 Variational SMC ELBO=-2.11450820e+04 at epoch 9
07/27/2018 10:01:18 Variational SMC ELBO=-2.91363574e+04 at epoch 10
07/27/2018 10:01:19 Variational SMC ELBO=-2.15191504e+04 at epoch 11
07/27/2018 10:01:19 Variational SMC ELBO=-2.16825312e+04 at epoch 12
07/27/2018 10:01:20 Variational SMC ELBO=-2.01211309e+04 at epoch 13
07/27/2018 10:01:20 Variational SMC ELBO=-2.19320332e+04 at epoch 14
07/27/2018 10:01:21 Variational SMC ELBO=-2

In [13]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.454539
SMC MSE at time 2: 2.208749
SMC MSE at time 3: 8.515728
SMC MSE at time 4: 0.024418
SMC MSE at time 5: 3.084924
SMC MSE at time 6: 1.415185
SMC MSE at time 7: 0.027037
SMC MSE at time 8: 1.435433
SMC MSE at time 9: 7.230900
SMC MSE at time 10: 0.067580
SMC MSE at time 11: 0.566357
SMC MSE at time 12: 0.599209
SMC MSE at time 13: 3.737939
SMC MSE at time 14: 3.796595
SMC MSE at time 15: 5.503882
SMC MSE at time 16: 0.533700
SMC MSE at time 17: 1.273374
SMC MSE at time 18: 1.540574
SMC MSE at time 19: 0.759202
SMC MSE at time 20: 2.954751
SMC MSE at time 21: 0.515373
SMC MSE at time 22: 3.671550
SMC MSE at time 23: 1.541496
SMC MSE at time 24: 1.304573
SMC MSE at time 25: 1.308868
SMC MSE at time 26: 2.138934
SMC MSE at time 27: 1.402532
SMC MSE at time 28: 0.070392
SMC MSE at time 29: 4.280766
SMC MSE at time 30: 2.287863
SMC MSE at time 31: 3.548551
SMC MSE at time 32: 0.725359
SMC MSE at time 33: 2.161368
SMC MSE at time 34: 0.576948
SMC MSE at time 35: 0.4

In [14]:
samples, elbos, inference = smc.particle_mh(num_particles, combinators.Model(ssm.init_ssm, theta=ssm_params), smc_runner, 500, 50, generative)

In [15]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 100]), 'sigma': Normal([torch.cuda.FloatTensor of size 100]), 'delta': Normal([torch.cuda.FloatTensor of size 100]), 'Z_0': Normal([torch.cuda.FloatTensor of size 100]), 'Z_1': Normal([torch.cuda.FloatTensor of size 100]), 'X_1': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 100]), 'X_2': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 100]), 'X_3': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 100]), 'X_4': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 100]), 'X_5': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 100]), 'X_6': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 100]), 'X_7': Normal([torch.cuda.FloatTensor of size 100x1x1]), 'Z_8': No

In [16]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [17]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 2.215430
SMC MSE at time 2: 0.003544
SMC MSE at time 3: 2.010585
SMC MSE at time 4: 6.860331
SMC MSE at time 5: 2.512908
SMC MSE at time 6: 0.793640
SMC MSE at time 7: 0.899639
SMC MSE at time 8: 0.298215
SMC MSE at time 9: 6.430851
SMC MSE at time 10: 3.191736
SMC MSE at time 11: 2.418517
SMC MSE at time 12: 3.818418
SMC MSE at time 13: 0.042156
SMC MSE at time 14: 11.515471
SMC MSE at time 15: 1.369139
SMC MSE at time 16: 0.845972
SMC MSE at time 17: 0.172040
SMC MSE at time 18: 0.705482
SMC MSE at time 19: 0.687381
SMC MSE at time 20: 1.607551
SMC MSE at time 21: 0.785513
SMC MSE at time 22: 0.228029
SMC MSE at time 23: 1.611311
SMC MSE at time 24: 0.820064
SMC MSE at time 25: 2.559157
SMC MSE at time 26: 3.083842
SMC MSE at time 27: 0.467193
SMC MSE at time 28: 0.158964
SMC MSE at time 29: 4.738479
SMC MSE at time 30: 0.555746
SMC MSE at time 31: 1.773221
SMC MSE at time 32: 0.378422
SMC MSE at time 33: 0.144900
SMC MSE at time 34: 0.672267
SMC MSE at time 35: 2.